## Data Types Check

In [1]:
import polars as pl

# Read the data from directory
df = pl.read_parquet("../0 - Data/merged_transactions.pq")

In [2]:
# Preview the columns to find out which columns needs to be adjusted
df.head()

User,Card,Year,Month,Day,Time,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?,CARD INDEX,Card Brand,Card Type,Card Number,Expires,CVV,Has Chip,Cards Issued,Credit Limit,Acct Open Date,Year PIN last Changed,Card on Dark Web,Person,Current Age,Retirement Age,Birth Year,Birth Month,Gender,Address,Apartment,City,State,Zipcode,Latitude,Longitude,Per Capita Income - Zipcode,Yearly Income - Person,Total Debt,FICO Score,Num Credit Cards
i64,i64,i64,i64,i64,str,str,str,i64,str,str,f64,i64,str,str,i64,str,str,i64,str,i64,str,i64,str,str,i64,str,str,i64,i64,i64,i64,str,str,i64,str,str,i64,f64,f64,str,str,str,i64,i64
0,0,2018,1,2,"""06:28""","""$130.95""","""Chip Transaction""",5817218446178736267,"""La Verne""","""CA""",91750.0,5912,null,"""No""",0,"""Visa""","""Debit""",4344676511950444,"""12/2022""",623,"""YES""",2,"""$24295""","""09/2002""",2008,"""No""","""Hazel Robinson""",53,66,1966,11,"""Female""","""462 Rose Lane""",null,"""La Verne""","""CA""",91750,34.15,-117.76,"""$29278""","""$59696""","""$127613""",787,5
0,0,2018,1,2,"""06:28""","""$130.95""","""Chip Transaction""",5817218446178736267,"""La Verne""","""CA""",91750.0,5912,null,"""No""",1,"""Visa""","""Debit""",4956965974959986,"""12/2020""",393,"""YES""",2,"""$21968""","""04/2014""",2014,"""No""","""Hazel Robinson""",53,66,1966,11,"""Female""","""462 Rose Lane""",null,"""La Verne""","""CA""",91750,34.15,-117.76,"""$29278""","""$59696""","""$127613""",787,5
0,0,2018,1,2,"""06:28""","""$130.95""","""Chip Transaction""",5817218446178736267,"""La Verne""","""CA""",91750.0,5912,null,"""No""",2,"""Visa""","""Debit""",4582313478255491,"""02/2024""",719,"""YES""",2,"""$46414""","""07/2003""",2004,"""No""","""Hazel Robinson""",53,66,1966,11,"""Female""","""462 Rose Lane""",null,"""La Verne""","""CA""",91750,34.15,-117.76,"""$29278""","""$59696""","""$127613""",787,5
0,0,2018,1,2,"""06:28""","""$130.95""","""Chip Transaction""",5817218446178736267,"""La Verne""","""CA""",91750.0,5912,null,"""No""",3,"""Visa""","""Credit""",4879494103069057,"""08/2024""",693,"""NO""",1,"""$12400""","""01/2003""",2012,"""No""","""Hazel Robinson""",53,66,1966,11,"""Female""","""462 Rose Lane""",null,"""La Verne""","""CA""",91750,34.15,-117.76,"""$29278""","""$59696""","""$127613""",787,5
0,0,2018,1,2,"""06:28""","""$130.95""","""Chip Transaction""",5817218446178736267,"""La Verne""","""CA""",91750.0,5912,null,"""No""",4,"""Mastercard""","""Debit (Prepaid)""",5722874738736011,"""03/2009""",75,"""YES""",1,"""$28""","""09/2008""",2009,"""No""","""Hazel Robinson""",53,66,1966,11,"""Female""","""462 Rose Lane""",null,"""La Verne""","""CA""",91750,34.15,-117.76,"""$29278""","""$59696""","""$127613""",787,5


In [3]:
# Combine 'Year', 'Month', 'Day', and 'Time' into a single 'Datetime' column
df = df.with_columns([
    # Ensure 'Year', 'Month', 'Day' are numeric first
    pl.col('Year').cast(pl.Int32).alias('Year'),
    pl.col('Month').cast(pl.Int32).alias('Month'),
    pl.col('Day').cast(pl.Int32).alias('Day'),
    
    # Ensure 'Time' is string for concatenation
    pl.col('Time').cast(pl.Utf8).alias('Time'),

    # Create a new 'Datetime_str' column by concatenating 'Year', 'Month', 'Day', and 'Time' as a string
    pl.concat_str(
        [pl.col("Year").cast(pl.Utf8), 
         pl.col("Month").cast(pl.Utf8).str.zfill(2), 
         pl.col("Day").cast(pl.Utf8).str.zfill(2), 
         pl.col("Time")],
        separator="-"
    ).alias("Datetime_str")
])

# Parse the concatenated 'Datetime_str' into a proper Datetime column
df = df.with_columns([
    pl.col("Datetime_str").str.strptime(pl.Datetime, format="%Y-%m-%d-%H:%M").alias("Datetime")
])

# Drop unnecessary columns: Year, Month, Day, Time, and temporary Datetime_str
df = df.drop(["Time", "Datetime_str"])

df.head()

User,Card,Year,Month,Day,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?,CARD INDEX,Card Brand,Card Type,Card Number,Expires,CVV,Has Chip,Cards Issued,Credit Limit,Acct Open Date,Year PIN last Changed,Card on Dark Web,Person,Current Age,Retirement Age,Birth Year,Birth Month,Gender,Address,Apartment,City,State,Zipcode,Latitude,Longitude,Per Capita Income - Zipcode,Yearly Income - Person,Total Debt,FICO Score,Num Credit Cards,Datetime
i64,i64,i32,i32,i32,str,str,i64,str,str,f64,i64,str,str,i64,str,str,i64,str,i64,str,i64,str,str,i64,str,str,i64,i64,i64,i64,str,str,i64,str,str,i64,f64,f64,str,str,str,i64,i64,datetime[μs]
0,0,2018,1,2,"""$130.95""","""Chip Transaction""",5817218446178736267,"""La Verne""","""CA""",91750.0,5912,null,"""No""",0,"""Visa""","""Debit""",4344676511950444,"""12/2022""",623,"""YES""",2,"""$24295""","""09/2002""",2008,"""No""","""Hazel Robinson""",53,66,1966,11,"""Female""","""462 Rose Lane""",null,"""La Verne""","""CA""",91750,34.15,-117.76,"""$29278""","""$59696""","""$127613""",787,5,2018-01-02 06:28:00
0,0,2018,1,2,"""$130.95""","""Chip Transaction""",5817218446178736267,"""La Verne""","""CA""",91750.0,5912,null,"""No""",1,"""Visa""","""Debit""",4956965974959986,"""12/2020""",393,"""YES""",2,"""$21968""","""04/2014""",2014,"""No""","""Hazel Robinson""",53,66,1966,11,"""Female""","""462 Rose Lane""",null,"""La Verne""","""CA""",91750,34.15,-117.76,"""$29278""","""$59696""","""$127613""",787,5,2018-01-02 06:28:00
0,0,2018,1,2,"""$130.95""","""Chip Transaction""",5817218446178736267,"""La Verne""","""CA""",91750.0,5912,null,"""No""",2,"""Visa""","""Debit""",4582313478255491,"""02/2024""",719,"""YES""",2,"""$46414""","""07/2003""",2004,"""No""","""Hazel Robinson""",53,66,1966,11,"""Female""","""462 Rose Lane""",null,"""La Verne""","""CA""",91750,34.15,-117.76,"""$29278""","""$59696""","""$127613""",787,5,2018-01-02 06:28:00
0,0,2018,1,2,"""$130.95""","""Chip Transaction""",5817218446178736267,"""La Verne""","""CA""",91750.0,5912,null,"""No""",3,"""Visa""","""Credit""",4879494103069057,"""08/2024""",693,"""NO""",1,"""$12400""","""01/2003""",2012,"""No""","""Hazel Robinson""",53,66,1966,11,"""Female""","""462 Rose Lane""",null,"""La Verne""","""CA""",91750,34.15,-117.76,"""$29278""","""$59696""","""$127613""",787,5,2018-01-02 06:28:00
0,0,2018,1,2,"""$130.95""","""Chip Transaction""",5817218446178736267,"""La Verne""","""CA""",91750.0,5912,null,"""No""",4,"""Mastercard""","""Debit (Prepaid)""",5722874738736011,"""03/2009""",75,"""YES""",1,"""$28""","""09/2008""",2009,"""No""","""Hazel Robinson""",53,66,1966,11,"""Female""","""462 Rose Lane""",null,"""La Verne""","""CA""",91750,34.15,-117.76,"""$29278""","""$59696""","""$127613""",787,5,2018-01-02 06:28:00


In [4]:
import polars as pl

# Clean the Amount and other financial columns before casting
df = df.with_columns([
    # Clean 'Amount' column
    pl.when(pl.col("Amount").is_not_null())
      .then(pl.col("Amount").str.replace('$', '', literal=True))  # Remove dollar signs and commas
      .otherwise(None)  # Handle potential nulls
      .cast(pl.Float64)  # Cast to Float64
      .alias("Amount"),
    
    # Clean 'Credit Limit' column
    pl.when(pl.col("Credit Limit").is_not_null())
      .then(pl.col("Credit Limit").str.replace('$', '', literal=True))
      .otherwise(None)
      .cast(pl.Float64)
      .alias("Credit Limit"),
    
    # Clean 'Yearly Income - Person' column
    pl.when(pl.col("Yearly Income - Person").is_not_null())
      .then(pl.col("Yearly Income - Person").str.replace('$', '', literal=True))
      .otherwise(None)
      .cast(pl.Float64)
      .alias("Yearly Income - Person"),
    
    # Clean 'Total Debt' column
    pl.when(pl.col("Total Debt").is_not_null())
      .then(pl.col("Total Debt").str.replace('$', '', literal=True))
      .otherwise(None)
      .cast(pl.Float64)
      .alias("Total Debt"),
    
    # Cast other columns directly
    pl.col("FICO Score").cast(pl.Int64).alias("FICO Score"),
    pl.col("Num Credit Cards").cast(pl.Int64).alias("Num Credit Cards")
])

# Display the head of the DataFrame to check the results
df.head()

User,Card,Year,Month,Day,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?,CARD INDEX,Card Brand,Card Type,Card Number,Expires,CVV,Has Chip,Cards Issued,Credit Limit,Acct Open Date,Year PIN last Changed,Card on Dark Web,Person,Current Age,Retirement Age,Birth Year,Birth Month,Gender,Address,Apartment,City,State,Zipcode,Latitude,Longitude,Per Capita Income - Zipcode,Yearly Income - Person,Total Debt,FICO Score,Num Credit Cards,Datetime
i64,i64,i32,i32,i32,f64,str,i64,str,str,f64,i64,str,str,i64,str,str,i64,str,i64,str,i64,f64,str,i64,str,str,i64,i64,i64,i64,str,str,i64,str,str,i64,f64,f64,str,f64,f64,i64,i64,datetime[μs]
0,0,2018,1,2,130.95,"""Chip Transaction""",5817218446178736267,"""La Verne""","""CA""",91750.0,5912,null,"""No""",0,"""Visa""","""Debit""",4344676511950444,"""12/2022""",623,"""YES""",2,24295.0,"""09/2002""",2008,"""No""","""Hazel Robinson""",53,66,1966,11,"""Female""","""462 Rose Lane""",null,"""La Verne""","""CA""",91750,34.15,-117.76,"""$29278""",59696.0,127613.0,787,5,2018-01-02 06:28:00
0,0,2018,1,2,130.95,"""Chip Transaction""",5817218446178736267,"""La Verne""","""CA""",91750.0,5912,null,"""No""",1,"""Visa""","""Debit""",4956965974959986,"""12/2020""",393,"""YES""",2,21968.0,"""04/2014""",2014,"""No""","""Hazel Robinson""",53,66,1966,11,"""Female""","""462 Rose Lane""",null,"""La Verne""","""CA""",91750,34.15,-117.76,"""$29278""",59696.0,127613.0,787,5,2018-01-02 06:28:00
0,0,2018,1,2,130.95,"""Chip Transaction""",5817218446178736267,"""La Verne""","""CA""",91750.0,5912,null,"""No""",2,"""Visa""","""Debit""",4582313478255491,"""02/2024""",719,"""YES""",2,46414.0,"""07/2003""",2004,"""No""","""Hazel Robinson""",53,66,1966,11,"""Female""","""462 Rose Lane""",null,"""La Verne""","""CA""",91750,34.15,-117.76,"""$29278""",59696.0,127613.0,787,5,2018-01-02 06:28:00
0,0,2018,1,2,130.95,"""Chip Transaction""",5817218446178736267,"""La Verne""","""CA""",91750.0,5912,null,"""No""",3,"""Visa""","""Credit""",4879494103069057,"""08/2024""",693,"""NO""",1,12400.0,"""01/2003""",2012,"""No""","""Hazel Robinson""",53,66,1966,11,"""Female""","""462 Rose Lane""",null,"""La Verne""","""CA""",91750,34.15,-117.76,"""$29278""",59696.0,127613.0,787,5,2018-01-02 06:28:00
0,0,2018,1,2,130.95,"""Chip Transaction""",5817218446178736267,"""La Verne""","""CA""",91750.0,5912,null,"""No""",4,"""Mastercard""","""Debit (Prepaid)""",5722874738736011,"""03/2009""",75,"""YES""",1,28.0,"""09/2008""",2009,"""No""","""Hazel Robinson""",53,66,1966,11,"""Female""","""462 Rose Lane""",null,"""La Verne""","""CA""",91750,34.15,-117.76,"""$29278""",59696.0,127613.0,787,5,2018-01-02 06:28:00


In [5]:
# Convert columns 'Is Fraud?', 'Has Chip', 'Card on Dark Web' to boolean
df = df.with_columns([
    pl.when(pl.col("Is Fraud?") == "Yes").then(1).otherwise(0).alias("Is Fraud"),
    pl.when(pl.col("Has Chip") == "YES").then(1).otherwise(0).alias("Has Chip"),
    pl.when(pl.col("Card on Dark Web") == "Yes").then(1).otherwise(0).alias("Card on Dark Web")
])
df.head()

User,Card,Year,Month,Day,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?,CARD INDEX,Card Brand,Card Type,Card Number,Expires,CVV,Has Chip,Cards Issued,Credit Limit,Acct Open Date,Year PIN last Changed,Card on Dark Web,Person,Current Age,Retirement Age,Birth Year,Birth Month,Gender,Address,Apartment,City,State,Zipcode,Latitude,Longitude,Per Capita Income - Zipcode,Yearly Income - Person,Total Debt,FICO Score,Num Credit Cards,Datetime,Is Fraud
i64,i64,i32,i32,i32,f64,str,i64,str,str,f64,i64,str,str,i64,str,str,i64,str,i64,i32,i64,f64,str,i64,i32,str,i64,i64,i64,i64,str,str,i64,str,str,i64,f64,f64,str,f64,f64,i64,i64,datetime[μs],i32
0,0,2018,1,2,130.95,"""Chip Transaction""",5817218446178736267,"""La Verne""","""CA""",91750.0,5912,null,"""No""",0,"""Visa""","""Debit""",4344676511950444,"""12/2022""",623,1,2,24295.0,"""09/2002""",2008,0,"""Hazel Robinson""",53,66,1966,11,"""Female""","""462 Rose Lane""",null,"""La Verne""","""CA""",91750,34.15,-117.76,"""$29278""",59696.0,127613.0,787,5,2018-01-02 06:28:00,0
0,0,2018,1,2,130.95,"""Chip Transaction""",5817218446178736267,"""La Verne""","""CA""",91750.0,5912,null,"""No""",1,"""Visa""","""Debit""",4956965974959986,"""12/2020""",393,1,2,21968.0,"""04/2014""",2014,0,"""Hazel Robinson""",53,66,1966,11,"""Female""","""462 Rose Lane""",null,"""La Verne""","""CA""",91750,34.15,-117.76,"""$29278""",59696.0,127613.0,787,5,2018-01-02 06:28:00,0
0,0,2018,1,2,130.95,"""Chip Transaction""",5817218446178736267,"""La Verne""","""CA""",91750.0,5912,null,"""No""",2,"""Visa""","""Debit""",4582313478255491,"""02/2024""",719,1,2,46414.0,"""07/2003""",2004,0,"""Hazel Robinson""",53,66,1966,11,"""Female""","""462 Rose Lane""",null,"""La Verne""","""CA""",91750,34.15,-117.76,"""$29278""",59696.0,127613.0,787,5,2018-01-02 06:28:00,0
0,0,2018,1,2,130.95,"""Chip Transaction""",5817218446178736267,"""La Verne""","""CA""",91750.0,5912,null,"""No""",3,"""Visa""","""Credit""",4879494103069057,"""08/2024""",693,0,1,12400.0,"""01/2003""",2012,0,"""Hazel Robinson""",53,66,1966,11,"""Female""","""462 Rose Lane""",null,"""La Verne""","""CA""",91750,34.15,-117.76,"""$29278""",59696.0,127613.0,787,5,2018-01-02 06:28:00,0
0,0,2018,1,2,130.95,"""Chip Transaction""",5817218446178736267,"""La Verne""","""CA""",91750.0,5912,null,"""No""",4,"""Mastercard""","""Debit (Prepaid)""",5722874738736011,"""03/2009""",75,1,1,28.0,"""09/2008""",2009,0,"""Hazel Robinson""",53,66,1966,11,"""Female""","""462 Rose Lane""",null,"""La Verne""","""CA""",91750,34.15,-117.76,"""$29278""",59696.0,127613.0,787,5,2018-01-02 06:28:00,0


In [6]:
# Ensure categorical or string columns are UTF8 type
df = df.with_columns([
    pl.col("User").cast(pl.Utf8).alias("User"),
    pl.col("Card").cast(pl.Utf8).alias("Card"),
    pl.col("Merchant Name").cast(pl.Utf8).alias("Merchant Name"),
    pl.col("Card Brand").cast(pl.Utf8).alias("Card Brand"),
    pl.col("Card Type").cast(pl.Utf8).alias("Card Type"),
    pl.col("Merchant City").cast(pl.Utf8).alias("Merchant City"),
    pl.col("Merchant State").cast(pl.Utf8).alias("Merchant State"),
    pl.col("Zip").cast(pl.Utf8).alias("Zip"),
    pl.col("Use Chip").cast(pl.Utf8).alias("Use Chip")
])

In [7]:
# Ensure 'Latitude' and 'Longitude' are float type
df = df.with_columns([
    pl.col("Latitude").cast(pl.Float64).alias("Latitude"),
    pl.col("Longitude").cast(pl.Float64).alias("Longitude")
])

In [8]:
# Find all the nulls within the data
df.null_count()

User,Card,Year,Month,Day,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?,CARD INDEX,Card Brand,Card Type,Card Number,Expires,CVV,Has Chip,Cards Issued,Credit Limit,Acct Open Date,Year PIN last Changed,Card on Dark Web,Person,Current Age,Retirement Age,Birth Year,Birth Month,Gender,Address,Apartment,City,State,Zipcode,Latitude,Longitude,Per Capita Income - Zipcode,Yearly Income - Person,Total Debt,FICO Score,Num Credit Cards,Datetime,Is Fraud
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,1632162,1730569,0,13200124,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9863188,0,0,0,0,0,0,0,0,0,0,0,0


In [9]:
# Fill null values with appropriate defaults
df = df.with_columns([
    pl.col("Zip").fill_null("Unknown"),
    pl.col("Errors?").fill_null(0),
    pl.col("Merchant State").fill_null("Unknown"),
    pl.col("Apartment").fill_null("Unknown")
])
df.head()

User,Card,Year,Month,Day,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?,CARD INDEX,Card Brand,Card Type,Card Number,Expires,CVV,Has Chip,Cards Issued,Credit Limit,Acct Open Date,Year PIN last Changed,Card on Dark Web,Person,Current Age,Retirement Age,Birth Year,Birth Month,Gender,Address,Apartment,City,State,Zipcode,Latitude,Longitude,Per Capita Income - Zipcode,Yearly Income - Person,Total Debt,FICO Score,Num Credit Cards,Datetime,Is Fraud
str,str,i32,i32,i32,f64,str,str,str,str,str,i64,str,str,i64,str,str,i64,str,i64,i32,i64,f64,str,i64,i32,str,i64,i64,i64,i64,str,str,str,str,str,i64,f64,f64,str,f64,f64,i64,i64,datetime[μs],i32
"""0""","""0""",2018,1,2,130.95,"""Chip Transaction""","""5817218446178736267""","""La Verne""","""CA""","""91750.0""",5912,"""0""","""No""",0,"""Visa""","""Debit""",4344676511950444,"""12/2022""",623,1,2,24295.0,"""09/2002""",2008,0,"""Hazel Robinson""",53,66,1966,11,"""Female""","""462 Rose Lane""","""Unknown""","""La Verne""","""CA""",91750,34.15,-117.76,"""$29278""",59696.0,127613.0,787,5,2018-01-02 06:28:00,0
"""0""","""0""",2018,1,2,130.95,"""Chip Transaction""","""5817218446178736267""","""La Verne""","""CA""","""91750.0""",5912,"""0""","""No""",1,"""Visa""","""Debit""",4956965974959986,"""12/2020""",393,1,2,21968.0,"""04/2014""",2014,0,"""Hazel Robinson""",53,66,1966,11,"""Female""","""462 Rose Lane""","""Unknown""","""La Verne""","""CA""",91750,34.15,-117.76,"""$29278""",59696.0,127613.0,787,5,2018-01-02 06:28:00,0
"""0""","""0""",2018,1,2,130.95,"""Chip Transaction""","""5817218446178736267""","""La Verne""","""CA""","""91750.0""",5912,"""0""","""No""",2,"""Visa""","""Debit""",4582313478255491,"""02/2024""",719,1,2,46414.0,"""07/2003""",2004,0,"""Hazel Robinson""",53,66,1966,11,"""Female""","""462 Rose Lane""","""Unknown""","""La Verne""","""CA""",91750,34.15,-117.76,"""$29278""",59696.0,127613.0,787,5,2018-01-02 06:28:00,0
"""0""","""0""",2018,1,2,130.95,"""Chip Transaction""","""5817218446178736267""","""La Verne""","""CA""","""91750.0""",5912,"""0""","""No""",3,"""Visa""","""Credit""",4879494103069057,"""08/2024""",693,0,1,12400.0,"""01/2003""",2012,0,"""Hazel Robinson""",53,66,1966,11,"""Female""","""462 Rose Lane""","""Unknown""","""La Verne""","""CA""",91750,34.15,-117.76,"""$29278""",59696.0,127613.0,787,5,2018-01-02 06:28:00,0
"""0""","""0""",2018,1,2,130.95,"""Chip Transaction""","""5817218446178736267""","""La Verne""","""CA""","""91750.0""",5912,"""0""","""No""",4,"""Mastercard""","""Debit (Prepaid)""",5722874738736011,"""03/2009""",75,1,1,28.0,"""09/2008""",2009,0,"""Hazel Robinson""",53,66,1966,11,"""Female""","""462 Rose Lane""","""Unknown""","""La Verne""","""CA""",91750,34.15,-117.76,"""$29278""",59696.0,127613.0,787,5,2018-01-02 06:28:00,0


In [10]:
# Check if all the null_counts are null
df.null_count()

User,Card,Year,Month,Day,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,Is Fraud?,CARD INDEX,Card Brand,Card Type,Card Number,Expires,CVV,Has Chip,Cards Issued,Credit Limit,Acct Open Date,Year PIN last Changed,Card on Dark Web,Person,Current Age,Retirement Age,Birth Year,Birth Month,Gender,Address,Apartment,City,State,Zipcode,Latitude,Longitude,Per Capita Income - Zipcode,Yearly Income - Person,Total Debt,FICO Score,Num Credit Cards,Datetime,Is Fraud
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [11]:
# Save the type checked data
df.write_parquet("../0 - Data/type_checked_transactions.pq")